<a href="https://colab.research.google.com/github/PARULCHUTANIPC/parul/blob/Paper-2/91-92-91%20Kaggle_dataset_upload_%2B_all_models%2C_cats_and_dogs_classificatoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install kaggle

from zipfile import ZipFile
import io, cv2, fnmatch, shutil, os, getpass, subprocess, random

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from time import time
from glob import glob
from sklearn.utils import class_weight
from google.colab import files

In [2]:
root_dir = '/content'
download_dir = os.path.join(root_dir,'cats_dogs')
data_dir = os.path.join(download_dir,'data')
train_path = os.path.join(data_dir,'train')
val_path = os.path.join(data_dir,'val')
train_dogs_path = os.path.join(train_path,'dogs')
train_cats_path = os.path.join(train_path,'cats')
val_dogs_path = os.path.join(val_path,'dogs')
val_cats_path = os.path.join(val_path,'cats')

data source - https://www.kaggle.com/c/dogs-vs-cats/data

In [3]:
!pip install kaggle

In [4]:
!mkdir .kaggle
!touch .kaggle/kaggle.json

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = root_dir
os.chdir(root_dir)
if 'kaggle.json' not in os.listdir(root_dir):downloaded = files.upload()
if 'cats_dogs' in os.listdir(root_dir):shutil.rmtree(download_dir)
os.mkdir(download_dir)
os.chdir(download_dir)
!kaggle competitions download -c dogs-vs-cats
!unzip -q -o train.zip
!unzip -q -o test1.zip

Saving kaggle.json to kaggle.json
 98% 267M/271M [00:04<00:00, 95.2MB/s]
100% 271M/271M [00:04<00:00, 62.8MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 88.7MB/s]
 99% 537M/543M [00:09<00:00, 83.5MB/s]
100% 543M/543M [00:09<00:00, 59.4MB/s]


In [6]:
cat_pattern = '*cat.*.jpg'
dog_pattern = '*dog.*.jpg'

images = glob('/content/cats_dogs/train/*.jpg', recursive=True)
cats = fnmatch.filter(images,cat_pattern)
dogs = fnmatch.filter(images,dog_pattern)

if 'data' not in os.listdir(download_dir):os.mkdir(data_dir)
if 'train' not in os.listdir(data_dir):os.mkdir(train_path)
if 'dogs' not in os.listdir(train_path):os.mkdir(train_dogs_path)
if 'cats' not in os.listdir(train_path):os.mkdir(train_cats_path)
  
if 'val' not in os.listdir(data_dir):os.mkdir(val_path)
if 'dogs' not in os.listdir(val_path):os.mkdir(val_dogs_path)
if 'cats' not in os.listdir(val_path):os.mkdir(val_cats_path)

for file in cats: shutil.copy2(file, train_cats_path)
for file in dogs: shutil.copy2(file, train_dogs_path)
  
  
# split train date into train and validation
train_len = len(os.listdir(train_dogs_path))
val_len = train_len * 0.3
val_dogs = random.sample(os.listdir(train_dogs_path),int(val_len))
val_cats = random.sample(os.listdir(train_cats_path),int(val_len))

for file in val_dogs:
  try: shutil.move(os.path.join(train_dogs_path,file), val_dogs_path)
  except: pass
for file in val_cats:
  try: shutil.move(os.path.join(train_cats_path,file), val_cats_path)
  except: pass
  
print(len(os.listdir(train_cats_path)))
print(len(os.listdir(val_cats_path)))

print(len(os.listdir(train_dogs_path)))
print(len(os.listdir(val_dogs_path)))

print('total train samples ', len(os.listdir(train_cats_path)) + len(os.listdir(train_dogs_path)))
print('total train samples ', len(os.listdir(val_cats_path)) + len(os.listdir(val_dogs_path)))

8750
3750
8750
3750
total train samples  17500
total train samples  7500


#CNN classifier

In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size=64
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

model = Sequential()
model.add(Conv2D(150, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(200, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(215, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(190, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=40,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
274/274 [==============================] - 244s 761ms/step - loss: 1.0684 - accuracy: 0.5189 - val_loss: 0.6423 - val_accuracy: 0.6435
Epoch 2/40
274/274 [==============================] - 205s 748ms/step - loss: 0.6556 - accuracy: 0.6378 - val_loss: 0.5441 - val_accuracy: 0.7212
Epoch 3/40
274/274 [==============================] - 206s 750ms/step - loss: 0.5699 - accuracy: 0.7102 - val_loss: 0.5870 - val_accuracy: 0.6757
Epoch 4/40
274/274 [==============================] - 206s 750ms/step - loss: 0.4837 - accuracy: 0.7667 - val_loss: 0.4269 - val_accuracy: 0.8101
Epoch 5/40
274/274 [==============================] - 205s 747ms/step - loss: 0.4448 - accuracy: 0.7957 - val_loss: 0.3920 - val_accuracy: 0.8293
Epoch 6/40
274/274 [==============================] - 206s 750ms/step - loss: 0.4031 - accuracy: 0.8202 - val_loss: 0.4185 - val_accuracy: 0.8033
Epoch 7/40
274/274 [==============================] - 205s 748ms/step - loss: 0.3641 - accuracy: 0.8394 - val_loss: 0.3149 -

#Transfer learning

In [ ]:
#BASE_MODEL = 'VGG16'
from keras.preprocessing.image import ImageDataGenerator

def transfer_learning(BASE_MODEL):
  if BASE_MODEL=='VGG16':
      from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
  elif BASE_MODEL=='vgg19':
      from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
  elif BASE_MODEL=='ResNet50':
      from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
  elif BASE_MODEL=='InceptionV3':
      from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
  elif BASE_MODEL=='Xception':
      from keras.applications.xception import Xception as PTModel, preprocess_input
  elif BASE_MODEL=='DenseNet169': 
      from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
  elif BASE_MODEL=='DenseNet121':
      from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
  else:
      raise ValueError('Unknown model: {}'.format(BASE_MODEL))
  
  import keras
  keras.backend.set_learning_phase(1)
  
  check_point_name = BASE_MODEL + '.model'
  model_weights = BASE_MODEL + '.h5'

  train_datagen = ImageDataGenerator(
          rescale=1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          preprocessing_function = preprocess_input)

  val_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function = preprocess_input)

  batch_size=32
  train_generator = train_datagen.flow_from_directory(
          '/content/cats_dogs/data/train',  # this is the target directory
          target_size=(250, 250),  # all images will be resized to 150x150
          batch_size=batch_size,
          class_mode='binary') 

  validation_generator = val_datagen.flow_from_directory(
          '/content/cats_dogs/data/val',
          target_size=(250, 250),
          batch_size=batch_size,
          class_mode='binary')    

  img_rows, img_cols, img_channel = 250, 250, 3
  base_model = PTModel(weights='imagenet'
                     ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

  add_model = Sequential()
  add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
  add_model.add(Dense(64, activation='relu'))
  add_model.add(Dense(1, activation='sigmoid'))

  model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


  for layer in base_model.layers:
      layer.trainable = False

      if layer.name.startswith('bn'):
          layer.call(layer.input, training=False)


  model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
                metrics=['accuracy'])
  start_time = time()
  model.fit_generator(
          train_generator,
          epochs=16,
          validation_data=validation_generator,
          #class_weight = class_weights,
          callbacks=[ModelCheckpoint(check_point_name, monitor='val_acc', save_best_only=True)])
  model.save_weights(model_weights)

  print('time taken ',time()-start_time)

In [ ]:
transfer_learning('VGG16')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
58892288/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/16
547/547 [==============================] - 382s 686ms/step - loss: 0.5134 - accuracy: 0.7450 - val_loss: 0.2836 - val_accuracy: 0.8903
Epoch 2/16
547/547 [==============================] - 369s 674ms/step - loss: 0.2856 - accuracy: 0.8861 - val_loss: 0.2221 - val_accuracy: 0.9137
Epoch 3/16
547/547 [==============================] - 368s 673ms/step - loss: 0.2358 - accuracy: 0.9093 - val_loss: 0.2043 - val_accuracy: 0.9189
Epoch 4/16
547/547 [==============================] - 366s 670ms/step - loss: 0.2152 - accuracy: 0.9163 - val_loss: 0.2001 - val_accuracy: 0.9192
Epoch 5/16
547/547 [==============================] - 366s 668ms/step - loss: 0.2038 - accuracy: 0.9216 - val_loss: 0.1992 - val_accuracy: 0.9112
Epoch 6/16
547/547 [==============================] - 367s 671ms/step - loss: 0.1880 - accuracy: 0.9271 - val_loss: 0.1754 - val_accuracy: 0.9301
Epoch 7/16
547/547 [==============================] - 367s 671ms/step - loss: 0.1793 - accuracy: 0.9295 - val_loss: 0.1663 -

In [ ]:
transfer_learning('DenseNet169')

In [ ]:
transfer_learning('vgg19')

In [ ]:
transfer_learning('InceptionV3')

In [ ]:
# running on TPU

# !pip install keras==2.1
# import tensorflow as tf
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense


BASE_MODEL = 'VGG16'

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='vgg19':
    from keras.applications.vgg19 import VGG19 as PTModel, preprocess_input
elif BASE_MODEL=='ResNet50':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))
    
    
import keras
keras.backend.set_learning_phase(1)
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        preprocessing_function = preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255,
                                preprocessing_function = preprocess_input)

batch_size=32
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(200, 200),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='binary')    

img_rows, img_cols, img_channel = 200, 200, 3
base_model = PTModel(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(64, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
    
    
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(loss='binary_crossentropy'
              ,optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator
        #class_weight = class_weights,
        ,callbacks=[ModelCheckpoint('Xception-transferlearning.model', monitor='val_acc', save_best_only=True)]
        )
model.save_weights('Xception.h5')

print('time taken ',time()-start_time)